In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import timeit

import tensorflow as tf
from tensorflow.keras import layers
keras = tf.keras
AUTOTUNE = tf.data.experimental.AUTOTUNE ## tf.data transformation parameters

from sklearn.model_selection import train_test_split

matplotlib.style.use('ggplot')

# Imports

In [2]:
# The number of celeb images: 2052599
n_celeb_images = 20000
# The format specification here left pads zeros on the number: 000006.
celeb_filenames = ['data/img_align_celeba/{:06d}.jpg'.format(i)
                    for i in range(1, n_celeb_images + 1)]
## slicing the array of strings, resulting in a set of scalar binary-strings
# path_ds = tf.data.Dataset.from_tensor_slices(celeb_filenames)

In [3]:
def jpg_image_to_array(image_path):
  """
  Loads JPEG image into 3D Numpy array of shape 
  (width, height, channels)
  """
  with Image.open(image_path) as image:         
    im_arr = np.fromstring(image.tobytes(), dtype=np.uint8)
    im_arr = im_arr.reshape((image.size[1], image.size[0], 3))                                   
    return im_arr

In [4]:
%%time
full_images=[]
for i in celeb_filenames:
    full_images.append(jpg_image_to_array(i))

/Users/AaronLee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  import sys


CPU times: user 18.8 s, sys: 5.8 s, total: 24.6 s
Wall time: 34.1 s


In [5]:
df_labels = pd.read_csv('data/list_attr_celeba.csv')

In [6]:
df_labels.columns = map(str.lower, df_labels.columns)
df_labels.replace([-1], 0, inplace=True)

In [7]:
df_labels.head()

,image_id,5_o_clock_shadow,arched_eyebrows,attractive,bags_under_eyes,bald,bangs,big_lips,big_nose,black_hair,...,sideburns,smiling,straight_hair,wavy_hair,wearing_earrings,wearing_hat,wearing_lipstick,wearing_necklace,wearing_necktie,young
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


## Functions

In [ ]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [218, 178])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [ ]:
## create a new dataset that loads and formats images on the fly
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

plt.figure(figsize = (8,8))
for n, image in enumerate(image_ds.take(4)):
    plt.subplot(2,2,n+1)
    plt.imshow(image)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

In [ ]:
print(image_ds)

In [ ]:
for n, image in enumerate(image_ds.take(20)):
    plt.imshow(image)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

# Covnet Tensorflow

In [ ]:
train_images=full_images[:90]
train_images = np.asarray(train_images)
train_labels=df_labels.male[:90]
train_labels = np.asarray(train_labels)

test_images=full_images[90:100]
test_images = np.asarray(test_images)
test_labels=df_labels.male[90:100]
test_labels = np.asarray(test_labels)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(full_images), np.asarray(df_labels.male)[:20000], shuffle=False)

In [9]:
X_train[0]

array([[[253, 231, 194],
        [253, 231, 194],
        [253, 231, 194],
        ...,
        [247, 226, 225],
        [254, 238, 222],
        [254, 238, 222]],

       [[253, 231, 194],
        [253, 231, 194],
        [253, 231, 194],
        ...,
        [249, 228, 225],
        [254, 238, 222],
        [254, 238, 222]],

       [[253, 231, 194],
        [253, 231, 194],
        [253, 231, 194],
        ...,
        [250, 231, 227],
        [255, 239, 223],
        [255, 239, 223]],

       ...,

       [[140,  74,  26],
        [116,  48,   1],
        [146,  78,  33],
        ...,
        [122,  55,  28],
        [122,  56,  30],
        [122,  56,  30]],

       [[130,  62,  15],
        [138,  70,  23],
        [166,  98,  53],
        ...,
        [118,  49,  20],
        [118,  51,  24],
        [118,  51,  24]],

       [[168, 100,  53],
        [204, 136,  89],
        [245, 177, 132],
        ...,
        [118,  49,  20],
        [120,  50,  24],
        [120,  50,  24]]

## First Layer

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(16, (5, 5), activation='relu',
                        input_shape=(218, 178, 3), padding = "same"),
    keras.layers.MaxPooling2D((2, 2))
])

In [ ]:
model.summary()

## Second Layer

In [ ]:
model.add(keras.layers.Conv2D(32, (5, 5), activation = 'relu', padding = 'same'))
model.add(keras.layers.MaxPool2D(2,2))

In [ ]:
model.summary()

## Third Layer

In [ ]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = "relu"))

In [ ]:
model.add(keras.layers.Dense(2,  activation = "softmax"))

In [ ]:
model.summary()

## Train the Network

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train

In [ ]:
%%time
X_train_images = X_train.reshape((X_train.shape[0], 218, 178, 3))
model.fit(X_train_images, y_train, epochs=1, verbose=2)

In [ ]:
%%time
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('\nTest accuracy {:5.2f}%'.format(100*test_acc))

## Save the model

## Plotting the Weights

In [ ]:
def plot_digit_weights(ax, digit):
    """Plot the weights from our fit fully connected network as an image."""
    digit_weigths = np.reshape(model.weights[0][:,digit], (218,178))
    ax.imshow(digit_weigths, cmap=plt.cm.winter, interpolation="nearest")    

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(12, 6))

for digit, ax in enumerate(axs.flatten()):
    plot_digit_weights(ax, digit)

remove_grid_lines(axs)
fig.suptitle("Fitted FC-weights plotted as an image")
plt.tight_layout()